##Introducción¶
Una red neuronal artificial (RNA) preentrenada es un modelo que ha sido entrenado previamente y luego distribuído (incluyendo arquitectura y valores de pesos) para su uso posterior. De esta manera no es necesario que volver a entrenarlo, o bien, se puede utilizar parte de el ya entrenado, y entrenar otras partes, realizando un ajuste fino o fine tuning. Constituye una manera de reducir los recursos y tiempos necesarios para generar un sistema de aprendizaje automático.

Hugging Face es una plataforma de aprendizaje automático, conocida por su librería de [Transformers](https://es.wikipedia.org/wiki/Transformador(modelo_de_aprendizajeautom%C3%A1tico)) creada para aplicaciones de procesamiento de lenguaje natural y que permite a los usuarios compartir conjuntos de datos y modelos de aprendizaje automático preentrenados. Su Model Hub contiene miles de modelos previamente entrenados de código abierto que cualquiera puede descargar y usar. Además dispone de varios cursos en los que se enseñan conceptos de aprendizaje automático a la vez que como utilizar sus herramientas.

Una de las formas más básicas de usar para inferencia la librería Hugging Face Transformers es la función pipeline(), que permite conectar uno de sus modelos de con los pasos necesarios para su preprocesamiento y posprocesamiento, abstrayendo la mayor parte del código complejo de la librería y ofreciendo una API simple dedicada a varias tareas, entre ellas, audio, visión artificial, procesamiento de lenguaje natural y tareas multimodales.

In [ ]:
from transformers import pipeline
# Avoid some warning
%env TOKENIZERS_PARALLELISM=false

##Análisis de sentimientos¶
El análisis de sentimientos premite clasificar textos de acuerdo a distintas etiquetas predeterminadas, como podrían ser textos positivos frente a negativos. Existen disintos modelos preentrenados disponibles para ello y para distintos idiomas, por ejemplo Spanish Sentiment Analysis Classifier

In [ ]:
# Allocate a piepeline for spanish sentiment-analysis
classifier = pipeline(model='VerificadoProfesional/SaBERT-Spanish-Sentiment-Analysis')
text_1 = "Nos alegra introducirles las redes neuronales artificiales preentrenadas a ustedes."
classifier(text_1)

In [ ]:
text_2 = "Sería una lástima que el tema no les resultara interesante."
classifier(text_2)

In [ ]:
# Showing the classifier model
print(classifier.model)
# classifier.model.summary()

... o también podemos realizar puntuaciones sobre los textos usando nlptown/bert-base-multilingual-uncased-sentiment.

In [ ]:
# Ranking texts
classifier = pipeline(model='nlptown/bert-base-multilingual-uncased-sentiment')
print(classifier(text_1))
print(classifier(text_2))

Podemos traducir los textos a inglés mediante Helsinki-NLP/opus-mt-es-en y realizar las puntuaciones nuevamente.

In [ ]:
# Translate the texts
translator = pipeline(model='Helsinki-NLP/opus-mt-es-en')
text_1_translated = translator(text_1)[0]['translation_text']
text_2_translated = translator(text_2)[0]['translation_text']

# And rank again
print(text_1_translated, classifier(text_1_translated))
print(text_2_translated, classifier(text_2_translated))

##Detección de objetos en imágenes¶
Detectar objetos en imágenes es una tarea clásica en visión por computador, y existen distintos modelos para ello, como DETR (End-to-End Object Detection) model with ResNet-50 backbone , estos modelos son más complejos que los utilizados anteriormente. Aquí lo probamos bajo una imagen extraída de Pixabay.

In [ ]:
# Some aditional library imports
import requests
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Download an image
url = "https://cdn.pixabay.com/photo/2021/12/23/05/27/still-life-6888656_960_720.jpg"
image_data = requests.get(url, stream=True).raw
image_sl = Image.open(image_data)

# Convert PIL Image, convert it to a format suitable for matplotlib
image_np = np.array(image_sl)

# Create a matplotlib figure and axis
fig, ax = plt.subplots(figsize=(12, 9))  # Width, height in inches
ax.imshow(image_np)

plt.show()

In [ ]:
# Allocate and execute a pipeline for object detection
object_detector = pipeline(model='facebook/detr-resnet-50')
predictions = object_detector(image_sl)

print("Number of detected objects:", len(predictions))
predictions

In [ ]:
# Create a matplotlib figure and axis
# from https://wellsr.com/python/zero-shot-object-detection-with-hugging-face-transformers/
fig, ax = plt.subplots(figsize=(12, 9))  # Width, height in inches
ax.imshow(image_np)

# Create a rectangle for each object
for prediction in predictions:
    box = prediction['box']
    label = prediction['label']
    score = prediction['score']
    xmin, ymin, xmax, ymax = box.values()
    rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=1, edgecolor='red', facecolor='none')

    # Add the rectangle to the axes
    ax.add_patch(rect)

    # Add text
    plt.text(xmin, ymin, f"{label}: {round(score,2)}", color='yellow', fontsize=10, verticalalignment='top')

# Display the image
plt.axis('off')
plt.show()

In [ ]:
# Showing the object detector model
print(object_detector.model)
#object_detector.model.summary()

##Imagen a texto¶
Generar texo a partir de una imagen otra de las tareas de visión por computador, puede suponer el detectar el texto dentro de una imagen (OCR de Optical Character Recognitio), con el modelo microsoft/trocr-base-handwritten probamos a reconocer texto escrito a mano.

In [ ]:
# Download an image
#url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02-00.jpg"
url = "https://bass-tian.com/es/img/Meticulosidad.jpg"
image_data = requests.get(url, stream=True).raw
image = Image.open(image_data)

#Convert PIL Image, convert it to a format suitable for matplotlib
image_np = np.array(image)

#Create a matplotlib figure and axis
fig, ax = plt.subplots(figsize=(12, 9))  # Width, height in inches
ax.imshow(image_np)

plt.show()

In [ ]:
# Allocate and execute a pipeline for ocr
ocr = pipeline(model='microsoft/trocr-base-handwritten')
ocr(image)

También se puede generar una leyenda que describa a una imagen, con el modelo nlpconnect/vit-gpt2-image-captioning probamos a reconocer el bodegón anterior.

In [ ]:
# Allocate and execute a pipeline for image caption
captioner = pipeline(model='nlpconnect/vit-gpt2-image-captioning')
captioner(image_sl)

##Generación de imágenes¶
Generar imágenes a partir de descripciones en lenguaje natural o prompts es una de las tareas de IA generativas más relevantes. Stable Diffusion es un modelo de código abierto para generar imágenes digitales de alta calidad a partir de descripciones en lenguaje natural. Basándonos en la implementación de Stable Diffusion en KerasCV se puede hacer llamadas a dicho modelo, aunque el tiempo de generación es bastante largo ...

In [ ]:
# import time
import keras_cv
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
model = keras_cv.models.StableDiffusion(img_width=512, img_height=512)

prompt = "photograph of a cowboy riding a bike"
images = model.text_to_image(prompt, batch_size=2)

plt.figure(figsize=(20, 20))
for i in range(len(images)):
    ax = plt.subplot(1, len(images), i + 1)
    plt.imshow(images[i])
    plt.axis("off")